In [1]:
import numpy as np
import json
from astropy.table import Table, vstack, join
from astropy.io import fits
import matplotlib.pyplot as plt

import sys
sys.path.insert(1, '/global/homes/h/hrincon/python_tools')
import VoidVolume as vol
import VoidOverlap as vo
import VoidCatalog as vc
import VoidSlicePlots as vsp

from vast.voidfinder.postprocessing import mknum

<frozen importlib._bootstrap>:241: RuntimeWarning: vast.voidfinder._voidfinder_cython_find_next.SpatialMap size changed, may indicate binary incompatibility. Expected 1848 from C header, got 2272 from PyObject


# Read in REVOLVER Void Catalogs

In [2]:
desi_gals_path = '/global/homes/h/hrincon/DESIVAST/galaxy_catalog/iron_ngc.fits'
sdss_gals_path = '/global/homes/h/hrincon/DESIVAST/alt_mag_cuts/SDSSK1/nsa_k1.fits'

# -20
desi_voids_path = '/global/homes/h/hrincon/DESIVAST/sdss_compare/CommonMask/DESI_V2_REVOLVER_Output.fits'
sdss_voids_path = '/global/homes/h/hrincon/DESIVAST/sdss_compare/CommonMask/SDSS_V2_REVOLVER_Output.fits'

# dimmer cut
desi_voids_path_dim = '/global/homes/h/hrincon/DESIVAST/sdss_compare/CommonMask/minus19.89/DESI_V2_REVOLVER_Output.fits'
sdss_voids_path_dim = '/global/homes/h/hrincon/DESIVAST/sdss_compare/CommonMask/minus19.94/SDSS_V2_REVOLVER_Output.fits'

# brighter cut
desi_voids_path_bright = '/global/homes/h/hrincon/DESIVAST/sdss_compare/CommonMask/minus20.06/DESI_V2_REVOLVER_Output.fits'
sdss_voids_path_bright = '/global/homes/h/hrincon/DESIVAST/sdss_compare/CommonMask/minus20.11/SDSS_V2_REVOLVER_Output.fits'


#DESI

v2r_desi = vc.V2Catalog(desi_voids_path)
#v2r_desi.add_galaxies(desi_gals_path) 
v2r_desi_dim = vc.V2Catalog(desi_voids_path_dim)
v2r_desi_bright = vc.V2Catalog(desi_voids_path_bright)

#SDSS
v2r_sdss = vc.V2Catalog(sdss_voids_path)
#v2r_sdss.add_galaxies(sdss_gals_path) 
v2r_sdss_dim = vc.V2Catalog(sdss_voids_path_dim)
v2r_sdss_bright = vc.V2Catalog(sdss_voids_path_bright)


In [3]:
masks_path = '/global/homes/h/hrincon/DESIVAST/galaxy_catalog/mask/alt_masks/masks.fits'

masks = fits.open(masks_path)

Catalog preparation

In [53]:
# Uncomment below lines if this is the first time running a catalog
# vfc.calculate_r_eff()
# vfc.calculate_vflag()
# vfc.calculate_r_eff()
#vfc_sdss.calculate_vflag()
#vfc_sdss.calculate_r_eff()

# Catalog General Properties

In [7]:
redtext = lambda x: f"\x1b[31m{x}\x1b[0m"

In [13]:
print('\n'+redtext("DESI REVOVLER"),'\n')
v2r_desi.void_stats()

print('\n'+redtext("SDSS REVOLVER"),'\n')
v2r_sdss.void_stats()

print('\n'+redtext("DESI REVOVLER dim"),'\n')
v2r_desi_dim.void_stats()

print('\n'+redtext("SDSS REVOLVER dim"),'\n')
v2r_sdss_dim.void_stats()

print('\n'+redtext("DESI REVOVLER bright"),'\n')
v2r_desi_bright.void_stats()

print('\n'+redtext("SDSS REVOLVER bright"),'\n')
v2r_sdss_bright.void_stats()


DESI REVOVLER 

79 voids
79 edge voids
0 interior voids
Mean Reff: 8.62 +/- 0.334 Mpc/h
Median Reff: 7.82 +/- 0.419 Mpc/h
Maximum Reff: 17.99 Mpc/h

SDSS REVOLVER 

102 voids
102 edge voids
0 interior voids
Mean Reff: 7.91 +/- 0.305 Mpc/h
Median Reff: 6.54 +/- 0.383 Mpc/h
Maximum Reff: 18.76 Mpc/h

DESI REVOVLER dim 

91 voids
91 edge voids
0 interior voids
Mean Reff: 8.52 +/- 0.297 Mpc/h
Median Reff: 7.82 +/- 0.372 Mpc/h
Maximum Reff: 16.27 Mpc/h

SDSS REVOLVER dim 

109 voids
108 edge voids
1 interior voids
Mean Reff: 7.88 +/- 0.279 Mpc/h
Median Reff: 6.83 +/- 0.35 Mpc/h
Maximum Reff: 17.81 Mpc/h

DESI REVOVLER bright 

74 voids
74 edge voids
0 interior voids
Mean Reff: 8.51 +/- 0.377 Mpc/h
Median Reff: 7.28 +/- 0.473 Mpc/h
Maximum Reff: 17.64 Mpc/h

SDSS REVOLVER bright 

87 voids
87 edge voids
0 interior voids
Mean Reff: 8.39 +/- 0.314 Mpc/h
Median Reff: 7.43 +/- 0.394 Mpc/h
Maximum Reff: 19.0 Mpc/h


# Mocks

In [4]:
desi_medians = []
for i in range (25):
    file = f'/global/homes/h/hrincon/DESIVAST/sdss_compare/CommonMask/altmtl/altmtl{i}_VoidFinder_Output.fits'
    v2r = vc.VoidFinderCatalog(file)
    median = np.median(v2r.maximals['r_eff'])
    desi_medians.append(median)
print(np.std(desi_medians))
desi_medians = []
for i in range (25):
    file = f'/global/homes/h/hrincon/DESIVAST/sdss_compare/CommonMask/altmtl/altmtl{i}_V2_REVOLVER_Output.fits'
    v2r = vc.V2Catalog(file)
    median = np.median(v2r.voids['radius'])
    desi_medians.append(median)
print(np.std(desi_medians))
desi_medians = []
for i in range (25):
    file = f'/global/homes/h/hrincon/DESIVAST/sdss_compare/CommonMask/altmtl/altmtl{i}_V2_VIDE_Output.fits'
    v2r = vc.V2Catalog(file)
    median = np.median(v2r.voids['radius'])
    desi_medians.append(median)
print(np.std(desi_medians))

0.46542123572689287
0.40343446892935075
0.6467667990112208


In [6]:

idx='000'
idx_list = [idx]
for i in range(24):
    idx = idx[0]+idx[1]+str(int(idx[2]) + 1)
    if int(idx[2]) == 4:
        idx = idx[0] +  str(int(idx[1]) + 1) + '0'

        if int(idx[1]) == 4:
            idx = str(int(idx[0]) + 1) + '00'
    idx_list.append(idx)

sdss_medians = []
for i in idx_list:
    file = f'/global/homes/h/hrincon/DESIVAST/sdss_compare/CommonMask/HR4/HR4_{i}_VoidFinder_Output.fits'
    v2r = vc.VoidFinderCatalog(file)
    median = np.median(v2r.maximals['r_eff'])
    sdss_medians.append(median)
print(np.std(sdss_medians))

sdss_medians = []
for i in idx_list:
    file = f'/global/homes/h/hrincon/DESIVAST/sdss_compare/CommonMask/HR4/HR4_{i}_V2_REVOLVER_Output.fits'
    v2r = vc.V2Catalog(file)
    median = np.median(v2r.voids['radius'])
    sdss_medians.append(median)
print(np.std(sdss_medians))

sdss_medians = []
for i in idx_list:
    file = f'/global/homes/h/hrincon/DESIVAST/sdss_compare/CommonMask/HR4/HR4_{i}_V2_VIDE_Output.fits'
    v2r = vc.V2Catalog(file)
    median = np.median(v2r.voids['radius'])
    sdss_medians.append(median)
print(np.std(sdss_medians))

0.4155288015773463
0.48307996143507764
0.5505844999558009


# Read in VIDE Void Catalogs

In [3]:
# -20
desi_voids_path = '/global/homes/h/hrincon/DESIVAST/sdss_compare/CommonMask/DESI_V2_VIDE_Output.fits'
sdss_voids_path = '/global/homes/h/hrincon/DESIVAST/sdss_compare/CommonMask/SDSS_V2_VIDE_Output.fits'

# dimmer cut
desi_voids_path_dim = '/global/homes/h/hrincon/DESIVAST/sdss_compare/CommonMask/minus19.89/DESI_V2_VIDE_Output.fits'
sdss_voids_path_dim = '/global/homes/h/hrincon/DESIVAST/sdss_compare/CommonMask/minus19.94/SDSS_V2_VIDE_Output.fits'

# brighter cut
desi_voids_path_bright = '/global/homes/h/hrincon/DESIVAST/sdss_compare/CommonMask/minus20.06/DESI_V2_VIDE_Output.fits'
sdss_voids_path_bright = '/global/homes/h/hrincon/DESIVAST/sdss_compare/CommonMask/minus20.11/SDSS_V2_VIDE_Output.fits'


#DESI

v2v_desi = vc.V2Catalog(desi_voids_path)
v2v_desi_dim = vc.V2Catalog(desi_voids_path_dim)
v2v_desi_bright = vc.V2Catalog(desi_voids_path_bright)

#SDSS
v2v_sdss = vc.V2Catalog(sdss_voids_path)
v2v_sdss_dim = vc.V2Catalog(sdss_voids_path_dim)
v2v_sdss_bright = vc.V2Catalog(sdss_voids_path_bright)


In [5]:
print('\n'+redtext("DESI VIDE"),'\n')
v2v_desi.void_stats()

print('\n'+redtext("SDSS VIDE"),'\n')
v2v_sdss.void_stats()

print('\n'+redtext("DESI VIDE dim"),'\n')
v2v_desi_dim.void_stats()

print('\n'+redtext("SDSS VIDE dim"),'\n')
v2v_sdss_dim.void_stats()

print('\n'+redtext("DESI VIDE bright"),'\n')
v2v_desi_bright.void_stats()

print('\n'+redtext("SDSS VIDE bright"),'\n')
v2v_sdss_bright.void_stats()


DESI VIDE 

17 voids
17 edge voids
0 interior voids
Mean Reff: 13.89 +/- 0.801 Mpc/h
Median Reff: 12.69 +/- 1.0 Mpc/h
Maximum Reff: 24.04 Mpc/h

SDSS VIDE 

17 voids
17 edge voids
0 interior voids
Mean Reff: 14.42 +/- 0.99 Mpc/h
Median Reff: 12.89 +/- 1.24 Mpc/h
Maximum Reff: 27.49 Mpc/h

DESI VIDE dim 

17 voids
17 edge voids
0 interior voids
Mean Reff: 13.89 +/- 0.902 Mpc/h
Median Reff: 12.84 +/- 1.13 Mpc/h
Maximum Reff: 27.06 Mpc/h

SDSS VIDE dim 

18 voids
18 edge voids
0 interior voids
Mean Reff: 13.72 +/- 0.65 Mpc/h
Median Reff: 13.25 +/- 0.815 Mpc/h
Maximum Reff: 21.41 Mpc/h

DESI VIDE bright 

19 voids
19 edge voids
0 interior voids
Mean Reff: 13.73 +/- 0.678 Mpc/h
Median Reff: 13.45 +/- 0.849 Mpc/h
Maximum Reff: 22.15 Mpc/h

SDSS VIDE bright 

18 voids
18 edge voids
0 interior voids
Mean Reff: 13.88 +/- 0.851 Mpc/h
Median Reff: 12.6 +/- 1.07 Mpc/h
Maximum Reff: 25.51 Mpc/h


In [6]:
desi_medians = []
for i in range (25):
    file = f'/global/homes/h/hrincon/DESIVAST/sdss_compare/CommonMask/altmtl/altmtl{i}_V2_VIDE_Output.fits'
    v2r = vc.V2Catalog(file)
    median = np.median(v2r.voids['radius'])
    desi_medians.append(median)
print(np.std(desi_medians))

0.6467667990112208


In [7]:
sdss_medians = []

idx='000'
idx_list = [idx]
for i in range(24):
    idx = idx[0]+idx[1]+str(int(idx[2]) + 1)
    if int(idx[2]) == 4:
        idx = idx[0] +  str(int(idx[1]) + 1) + '0'

        if int(idx[1]) == 4:
            idx = str(int(idx[0]) + 1) + '00'
    idx_list.append(idx)

for i in idx_list:
    file = f'/global/homes/h/hrincon/DESIVAST/sdss_compare/CommonMask/HR4/HR4_{i}_V2_VIDE_Output.fits'
    v2r = vc.V2Catalog(file)
    median = np.median(v2r.voids['radius'])
    sdss_medians.append(median)
print(np.std(sdss_medians))

0.5505844999558009


# Read in VoidFinder Voids (WIP)

In [4]:
# -20
desi_voids_path = '/global/homes/h/hrincon/DESIVAST/sdss_compare/CommonMask/DESI_VoidFinder_Output.fits'
sdss_voids_path = '/global/homes/h/hrincon/DESIVAST/sdss_compare/CommonMask/SDSS_VoidFinder_Output.fits'

# dimmer cut
desi_voids_path_dim = '/global/homes/h/hrincon/DESIVAST/sdss_compare/CommonMask/minus19.89/DESI_VoidFinder_Output.fits'
sdss_voids_path_dim = '/global/homes/h/hrincon/DESIVAST/sdss_compare/CommonMask/minus19.94/SDSS_VoidFinder_Output.fits'

# brighter cut
desi_voids_path_bright = '/global/homes/h/hrincon/DESIVAST/sdss_compare/CommonMask/minus20.06/DESI_VoidFinder_Output.fits'
sdss_voids_path_bright = '/global/homes/h/hrincon/DESIVAST/sdss_compare/CommonMask/minus20.11/SDSS_VoidFinder_Output.fits'


#DESI

vfc_desi = vc.VoidFinderCatalog(desi_voids_path)
vfc_desi_dim = vc.VoidFinderCatalog(desi_voids_path_dim)
vfc_desi_bright = vc.VoidFinderCatalog(desi_voids_path_bright)

#SDSS
vfc_sdss = vc.VoidFinderCatalog(sdss_voids_path)
vfc_sdss_dim = vc.VoidFinderCatalog(sdss_voids_path_dim)
vfc_sdss_bright = vc.VoidFinderCatalog(sdss_voids_path_bright)


In [5]:
vfc_desi.calculate_r_eff()
vfc_desi_dim.calculate_r_eff()
vfc_desi_bright.calculate_r_eff()
vfc_sdss.calculate_r_eff()
vfc_sdss_dim.calculate_r_eff()
vfc_sdss_bright.calculate_r_eff()


In [8]:
print('\n'+redtext("DESI VIDE"),'\n')
vfc_desi.void_stats()

print('\n'+redtext("SDSS VIDE"),'\n')
vfc_sdss.void_stats()

print('\n'+redtext("DESI VIDE dim"),'\n')
vfc_desi_dim.void_stats()

print('\n'+redtext("SDSS VIDE dim"),'\n')
vfc_sdss_dim.void_stats()

print('\n'+redtext("DESI VIDE bright"),'\n')
vfc_desi_bright.void_stats()

print('\n'+redtext("SDSS VIDE bright"),'\n')
vfc_sdss_bright.void_stats()


DESI VIDE 

29 voids
29 edge voids
0 near-edge voids
0 interior voids
Mean Reff: 13.92 +/- 0.366 Mpc/h
Median Reff: 13.6 +/- 0.459 Mpc/h
Maximum Reff: 19.32 Mpc/h

SDSS VIDE 

28 voids
28 edge voids
0 near-edge voids
0 interior voids
Mean Reff: 13.72 +/- 0.412 Mpc/h
Median Reff: 12.9 +/- 0.517 Mpc/h
Maximum Reff: 19.92 Mpc/h

DESI VIDE dim 

33 voids
31 edge voids
2 near-edge voids
0 interior voids
Mean Reff: 13.4 +/- 0.315 Mpc/h
Median Reff: 12.93 +/- 0.394 Mpc/h
Maximum Reff: 18.43 Mpc/h

SDSS VIDE dim 

28 voids
27 edge voids
0 near-edge voids
1 interior voids
Mean Reff: 13.64 +/- 0.359 Mpc/h
Median Reff: 13.14 +/- 0.45 Mpc/h
Maximum Reff: 18.35 Mpc/h

DESI VIDE bright 

31 voids
31 edge voids
0 near-edge voids
0 interior voids
Mean Reff: 13.64 +/- 0.404 Mpc/h
Median Reff: 13.4 +/- 0.507 Mpc/h
Maximum Reff: 20.36 Mpc/h

SDSS VIDE bright 

31 voids
31 edge voids
0 near-edge voids
0 interior voids
Mean Reff: 14.09 +/- 0.364 Mpc/h
Median Reff: 13.66 +/- 0.457 Mpc/h
Maximum Reff: 19.32

In [12]:
desi_medians = []
for i in range (25):
    file = f'/global/homes/h/hrincon/DESIVAST/sdss_compare/CommonMask/altmtl/altmtl{i}_VoidFinder_Output.fits'
    vfc = vc.VoidFinderCatalog(file)
    vfc.calculate_r_eff()
    median = np.median(vfc.maximals['r_eff'])
    desi_medians.append(median)
print(np.std(desi_medians))

0.46542123572689287


In [10]:
sdss_medians = []

idx='000'
idx_list = [idx]
for i in range(24):
    idx = idx[0]+idx[1]+str(int(idx[2]) + 1)
    if int(idx[2]) == 4:
        idx = idx[0] +  str(int(idx[1]) + 1) + '0'

        if int(idx[1]) == 4:
            idx = str(int(idx[0]) + 1) + '00'
    idx_list.append(idx)

for i in idx_list:
    file = f'/global/homes/h/hrincon/DESIVAST/sdss_compare/CommonMask/HR4/HR4_{i}_VoidFinder_Output.fits'
    vfc = vc.VoidFinderCatalog(file)
    vfc.calculate_r_eff()
    median = np.median(vfc.maximals['r_eff'])
    sdss_medians.append(median)
print(np.std(sdss_medians))

0.4155288015773463
